In [1]:
from backbone.resnet_2d3d import resnet18_2d3d_simplified

resModel = resnet18_2d3d_simplified()
print(resModel)

ResNet2d3d_full(
  (conv1): Conv3d(1, 64, kernel_size=(1, 5, 5), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock2d(
      (conv1): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (layer2): Sequential(
    (0): BasicBlock2d(
      (conv1): Conv3d(64, 128, kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), bias=False)
      (bn1): BatchNorm3d(1

In [2]:
import torch

test_tensor = torch.randn(96, 1, 5, 30, 60)
output = resModel(test_tensor)
# Print the output size
print(output.size())

torch.Size([96, 128, 2, 1, 2])


In [1]:
from helpers.dataset import BldgDataset

dataset = BldgDataset("./DemoExperiment/1117/demo_test.zip")

Loading demo: 100%|██████████| 300/300 [00:01<00:00, 247.04path/s]


In [9]:
import numbers
import random


class RandomCropWithProb:
    def __init__(self, size, p=0.8, consistent=True):
        if isinstance(size, numbers.Number):
            self.size = (int(size), int(size))
        else:
            self.size = (size[0], size[1])
        self.consistent = consistent
        self.threshold = p

    def __call__(self, imgmap):
        img1 = imgmap[0]
        h, w = img1.shape
        print(w)
        if self.size is not None:
            th, tw = self.size
            print(tw)
            print(w - tw)
            if w == tw and h == th:
                return imgmap
            if self.consistent:
                if random.random() < self.threshold:
                    x1 = random.randint(0, w - tw)
                    y1 = random.randint(0, h - th)
                else:
                    x1 = int(round((w - tw) / 2.0))
                    y1 = int(round((h - th) / 2.0))
                return [i[y1 : y1 + th, x1 : x1 + tw] for i in imgmap]
            else:
                result = []
                for i in imgmap:
                    if random.random() < self.threshold:
                        x1 = random.randint(0, w - tw)
                        y1 = random.randint(0, h - th)
                    else:
                        x1 = int(round((w - tw) / 2.0))
                        y1 = int(round((h - th) / 2.0))
                    result.append(i[y1 : y1 + th, x1 : x1 + tw])
                return result
        else:
            return imgmap

In [11]:
import matplotlib.pyplot as plt

%matplotlib inline
test_data = dataset[0]["imgs"][:10]
cropped_transform = RandomCropWithProb(size=[20, 40], p=1.0)
cropped_data = cropped_transform(test_data)
for i in range(len(cropped_data)):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    img = cropped_data[i]
    ax.set_title(f"img {i}")
    ax.imshow(img.reshape(20, 40), cmap="gray", vmin=0, vmax=255)
    plt.show()


60
40
20


C:\Users\moumo\AppData\Local\Temp\ipykernel_42968\4032209050.py:12: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
import os
import shutil
import zipfile


def preprocess_image_data(base_path, zip_name):
    # Unzip the file
    zip_path = os.path.join(base_path, zip_name)
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(base_path)

    # Iterate through each 'pathXX' folder and rename PNGs using a temporary name
    for i in range(300):
        folder_name = f"path{str(i).zfill(2)}"
        folder_path = os.path.join(base_path, folder_name)

        # First, rename to temporary names
        for j in range(30):
            old_name = f"panoramic_{str(j).zfill(2)}.png"
            temp_name = f"temp_{str(j).zfill(2)}.png"
            old_path = os.path.join(folder_path, old_name)
            temp_path = os.path.join(folder_path, temp_name)

            if os.path.exists(old_path):
                os.rename(old_path, temp_path)

        # Then, rename temporary files to the final names
        for j in range(30):
            temp_name = f"temp_{str(j).zfill(2)}.png"
            new_name = f"panoramic_{str(29-j).zfill(2)}.png"
            temp_path = os.path.join(folder_path, temp_name)
            new_path = os.path.join(folder_path, new_name)

            if os.path.exists(temp_path):
                os.rename(temp_path, new_path)

        print(f"Finished processing folder {folder_name}")

    # Rezip the folders with the new name
    new_zip_name_parts = zip_name[:-4].split("_")  # Split the original zip name
    new_zip_name_parts[1], new_zip_name_parts[2] = (
        new_zip_name_parts[2],
        new_zip_name_parts[1],
    )  # Swap 'start' and 'end'
    new_zip_name = "_".join(new_zip_name_parts) + ".zip"

    with zipfile.ZipFile(os.path.join(base_path, new_zip_name), "w") as zipf:
        for i in range(300):
            folder_name = f"path{str(i).zfill(2)}"
            folder_path = os.path.join(base_path, folder_name)
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    zipf.write(
                        os.path.join(root, file),
                        os.path.relpath(os.path.join(root, file), base_path),
                    )

    # Optional: Delete the path folders after re-zipping
    for i in range(300):
        shutil.rmtree(os.path.join(base_path, f"path{str(i).zfill(2)}"))


# Usage
preprocess_image_data("./DemoExperiment/process", "demo_corridor_room_a.zip")